# Assignment2 - Logistic Regression 구현
none으로 구성된 영역에 대해서 채워보았습니다. 

In [1]:
import math, random
from functools import partial,reduce
from assignment2 import *
import pandas as pd
from sklearn.model_selection import train_test_split

data 설명
1) Label: 유료 계정 등록 여부(target)
2) bias: 회귀 모형에서의 상수항을 위한 term (추정 시 포함하지 않아도 ok)
3) experience: 근속연수
4) salary: 연봉

어떤 사용자가 유료 계정을 등록할지(Label == 1)에 대한 예측을 로지스틱 회귀 모형으로 진행합니다.


In [2]:
data = pd.read_csv('assignment_2.csv')

In [3]:
data

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000
...,...,...,...,...
195,0,1,6.5,84000
196,0,1,6.9,73000
197,0,1,5.1,72000
198,1,1,9.1,69000


### 0. Gradient Descent

In [4]:
#0이될때 가장 작은 지점이라고 볼 수 있다. 따라서 벡터 v가 direction으로 step_size 만큼 이동한 것을 v2, direction2으로 지정하는 함수를 만듭니다.

def step(v, direction, step_size):
    return [v2 + step_size * direction2 for v2, direction2 in zip(v,direction)]

In [5]:
def safe(f) :
    """
    f에 대한 예외처리를 위한 함수(f가 infinite일 때)
    """
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')
    return safe_f

In [6]:
# 가장 적합한 모델을 찾기위해서는 1)모델의 오류를 최소화하거나 2)우도를 최대화 하는 방법이 있습니다.
# 이를위해 경사하강법 gradient을 사용합니다.
#즉, 함수가 가장 빠르게 증가할 수 있는 방향을 찾아야 하는데 임의의 시작점에서 gradient를 계산하고 그 gradient방향으로 조금씩 이동하는 것을 반복해야합니다.
#목적함수를 최소화시키는 theta를 경사 하강법을 사용해서 찾는다.

def minimize_bgd(target_fn, gradient_fn, theta_0, tolerance = 0.00001): # bgd: batch gradient descent
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    
    # 시작점 설정
    theta = theta_0 #시작점
    target_fn = safe(target_fn) # 오류를 처리할 수 있도록 target_fn에 safe함수를 사용하여 변환합니다
    value = target_fn(theta) # 최소화시키려는 값을 value로하여 target_fn에 theta를 넣습니다
    
    while True:
        gradient = gradient_fn(theta) # gradient값을 계산합니다
        next_thetas = [step(theta,gradient, -step_size) for step_size in step_sizes] #### update thetas --> 각 step sizes에 따른 theta값을 list형태로 리턴
        #next_thetas는 theta가 gradient방향으로 -step_size(점점작아지도록)만큼 이동하도록 합니다. 
        # 함수를 최소화시키는 theta 선택
        obj = next_thetas #obj라는 이름으로 next_thetas를 지정합니다. 
        #print(obj)
        key = target_fn #key라는 이름으로 target_fn을 지정합니다. 
        #print(key)
        next_theta = min(obj, key = key) #next_thetas가 최소화되는 방향으로 이동한 것을 다시 next_theta로 지정합니다 
        #print(next_theta)
        next_value = target_fn(next_theta) # next_theta에서 최소화 되는 지점을 next_value라고 지정합니다. 
        #print(next_value)
        
        # tolerance만큼 수렴하면 멈춤
        temp = abs(next_value - value) #오류값이 우리가 설정한 tolerance(용인가능한 범위)보다 작을경우경우 반복문에서 벗어나고자합니다. 
        
        # temp가 음수일 경우도 있기 때문에 절대값을 씌어 주어야 합니다. 
        if temp < tolerance:
            return theta #이 지점에서의 theta를 return합니다 
        else: 
            theta, value = next_theta, next_value #temp가 tolerance보다 클경우 다시 이동하기 위하여 다시 새로운 theta, value로 돌아가야 합니다

In [7]:
def stochastic():
    """
    sgd 구현 (추가적인 부분이니 필수는 아닙니다.)
    random sampling 하는 부분(함수로 따로 구현하셔도 ok) + gd 부분
    """
    #죄송합니다...

### 1. 로지스틱 함수
해당 함수는 1/(1+exp[-(ax+b)]로 표현되었음을 기억합시다.# 1. 로지스틱

In [8]:
def logistic(x):
    try:
        return 1.0 / (1 + math.exp(-x))
    except OverflowError:
        return 1e-9 #OverflowError함수를 사용하여 예외처리를 해주었습니다. 

In [9]:
def softmax():
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
#softmax 정의이기때문에...

### 2. Likelihood 구현
그냥 Likelihood function 대신, log likelihood function을 이용해서 구현하세요.

In [10]:
# Likelihood 의 미분해서 0이 되는 값을 찾아야 하는데, 복잡하기 때문에 Log를 취하며 이것을 Log Likelihood라고 합니다. 
def logistic_log_likelihood_i(x_i, y_i, beta): # 개별 데이터포인트에 대한 likelihood 값
    if y_i == 1:
        return math.log(logistic(dot(x_i, beta)))
        #likelihood에 log를 씌우고 y_i이 1이나올경우를 x_i에 beta의 곱에 대한 결과에 log를 씌우고
    else:
        return math.log(1 - logistic(dot(x_i, beta))+1e-9)
    #나머지의 경우를 로그를 씌웁니다.


In [11]:
def logistic_log_likelihood(x,y,beta): # 전체 데이터에 대한 likelihood
    return sum(logistic_log_likelihood_i(x_i, y_i, beta)
               for x_i, y_i in zip(x, y))
#같은 방식으로 데이터 전체에 대한 전체합에 대한 우도를 로그를 씌웁니다. 

In [12]:
# 3번부터는 공지방의 소스코드 활용

### 3. Gradient for Log Reg
아래 3가지 함수에 대해 해당 함수의 인자와 기능을 자세히 설명하세요.

In [13]:
def logistic_log_partial_ij(x_i, y_i, beta, j):
    """
    특정 데이터 x_i에 대한 beta_j의 gradient 값 --> 상수 형태의 리턴값
    """
    return (y_i - logistic(dot(x_i, beta))) * x_i[j] # j: beta의 index

def logistic_log_gradient_i(x_i, y_i, beta):
    """
    x_i(row)에 대한 beta의 gradient 값(column 개수만큼) --> 리스트 형태의 리턴값을 가짐
    """
    return [logistic_log_partial_ij(x_i, y_i, beta, j) for j, _ in enumerate(beta)]

def logistic_log_gradient(x, y, beta):
    """
    전체 데이터에 대한 beta의 gradient 값 
    --> 이중 리스트를 vector_add 시켜서 beta_j에 대한 gradient값들을 rowSum 해놓은 리스트를 리턴
    """
    return reduce(vector_add, [logistic_log_gradient_i(x_i, y_i, beta) for x_i, y_i in zip(x,y)])
# reduce ~ == np.sum(iter, axis = 0)

### 4. Model Fitting
위에서 구현한 log likelihood를 이용하여 Model을 Fitting 시켜보세요.  
앞서 우리는 log likelihood를 maximize하는 방향으로 회귀계수를 추정한다고 배웠습니다.  
Gradient Descent는 경사 "하강법"으로 최솟값을 찾는 데에 사용되는 알고리즘입니다.  
따라서 log likelihood를 적절히 변형을 해야 Gradient Descent 코드를 적용할 수 있습니다.  
log likelihood 변형 함수는 assignment2.py에 구현되어있으니, None값만 채워주시면 됩니다.

In [14]:
data

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000
...,...,...,...,...
195,0,1,6.5,84000
196,0,1,6.9,73000
197,0,1,5.1,72000
198,1,1,9.1,69000


In [15]:
X = data.drop('Label', axis = 1)
y = data['Label']

In [16]:
X=X.as_matrix() # matrix로 변환해주어야 합니다!
y=y.as_matrix()

C:\Users\gjwjd\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\gjwjd\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [17]:
X[:10] ## X: 데이터 스케일이 필요함.

array([[1.0e+00, 7.0e-01, 4.8e+04],
       [1.0e+00, 1.9e+00, 4.8e+04],
       [1.0e+00, 2.5e+00, 6.0e+04],
       [1.0e+00, 4.2e+00, 6.3e+04],
       [1.0e+00, 6.0e+00, 7.6e+04],
       [1.0e+00, 6.5e+00, 6.9e+04],
       [1.0e+00, 7.5e+00, 7.6e+04],
       [1.0e+00, 8.1e+00, 8.8e+04],
       [1.0e+00, 8.7e+00, 8.3e+04],
       [1.0e+00, 1.0e+01, 8.3e+04]])

In [18]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_scaled = ss.fit_transform(data.drop(['Label'], axis = 1)) # X_scaled


In [19]:
for i,_ in enumerate(X_scaled):
    X_scaled[i][0] = 1 # bias term 추가

In [20]:
from sklearn.model_selection import train_test_split
import random

In [21]:
random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y)

In [22]:
# 아래에 Model Fitting 진행
# partial을 이용해 fn과 gradient_fn 구현

fn = partial(logistic_log_likelihood, x_train, y_train)
gradient_fn = partial(logistic_log_gradient, x_train, y_train)

beta_0 = [random.random() for _ in range(3)] # 임의의 시작점
#beta_0 = [1,1,1]

# 경사 하강법으로 최적화
beta_hat = minimize_bgd(neg(fn), neg_all(gradient_fn), beta_0)


In [23]:
beta_hat

[-1.9572843730755123, 4.4450391911094265, -4.104964259568767]